https://github.com/dmlc/xgboost

https://github.com/Microsoft/LightGBM

In [3]:
def read_data():
    X, y = [], []
    with open("data/prepared_data", "r") as f:
        for i in f.readlines():
            s = i.replace("\n", "")
            s_split = s.split(",")
            X.append([float(j) for j in s_split[0:-1]])
            y.append(s_split[-1])
            
    return X, y

X, y = read_data()

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

log_reg = LogisticRegression()
rf = RandomForestClassifier()
nb = GaussianNB()

In [4]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(rf, X, y).mean())
print(cross_val_score(log_reg, X, y).mean())
print(cross_val_score(nb, X, y).mean())

0.823559884257
0.768519999815
0.689019703345


In [5]:
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [13]:
def keras_perceptron_plain(X, y):
    
    num_labels = max(y) + 1
    num_input = X[0].shape[1]
    print (num_labels)
    return

    tf.reset_default_graph()
    sess = tf.InteractiveSession()

    model = Sequential()
    model.add(Dense((num_input + num_labels) / 2, activation='sigmoid', input_dim=num_input))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(X, y, epochs=30, batch_size=32)

    return model

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
model = keras_perceptron_plain(X_train, y_train)

scores = model.evaluate(X_test, y_test, verbose=0)

print (scores)
print("Accuracy: %.2f%%" % (scores[1]*100))

TypeError: Can't convert 'int' object to str implicitly

In [9]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_accuracy(pred, y_test):
    count_corr = 0
    for i in zip(y_test, pred):
        if i[0] == i[1]:
            count_corr += 1
        
    return count_corr, (count_corr/len(y_test))

def get_metrics(predicted, correct):
    count_corr, accuracy = get_accuracy(predicted, y_test)
    print ("all: " + str(len(predicted)) + " " + "correct: " + str(count_corr))
    print ("accuracy" + str(accuracy))
    print ("precision " + str(precision_score(correct, predicted)))
    print ("recall " + str(recall_score(correct, predicted)))
    print ("f1 " + str(f1_score(correct, predicted)))

In [10]:
labels_pred = model.predict(X_test)

In [12]:
print (labels_pred)

[[ 0.]
 [ 0.]
 [ 0.]
 ..., 
 [ 0.]
 [ 0.]
 [ 0.]]


In [11]:
labels_pred_new = []

for i in labels_pred:
    proba_m = i[0]
    proba_f = 1 - i[0]
    
    if proba_m > proba_f:
        labels_pred_new.append(0)
    else:
        labels_pred_new.append(1)
        
get_metrics(labels_pred_new, y_test)

all: 16500 correct: 0
accuracy0.0


ValueError: Mix of label input types (string and number)